In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

import keras.backend as k
import keras
from keras.models import Model
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.metrics import categorical_crossentropy
from keras.models import load_model

Using TensorFlow backend.


In [2]:
model = load_model(r'E:\tonyStark\NeuralNetworks\gesture\best.h5')

In [ ]:
np.set_printoptions(suppress=True)
def empty(a):
    pass

cap = cv2.VideoCapture(0)

cv2.namedWindow("Trackbars")    
cv2.resizeWindow("Trackbars", 640, 320)
cv2.createTrackbar("hMin", "Trackbars", 53, 179, empty)     
cv2.createTrackbar("hMax", "Trackbars", 140, 179, empty)
cv2.createTrackbar("sMin", "Trackbars", 57, 255, empty)
cv2.createTrackbar("sMax", "Trackbars", 192, 255, empty)
cv2.createTrackbar("vMin", "Trackbars", 67, 255, empty)
cv2.createTrackbar("vMax", "Trackbars", 166, 255, empty)

while True:
    succ, img = cap.read()
    cv2.rectangle(img, (300,100), (500,300), (255,255,255), 1)
    
    hMin = cv2.getTrackbarPos("hMin", "Trackbars")      
    hMax = cv2.getTrackbarPos("hMax", "Trackbars")
    sMin = cv2.getTrackbarPos("sMin", "Trackbars")
    sMax = cv2.getTrackbarPos("sMax", "Trackbars")
    vMin = cv2.getTrackbarPos("vMin", "Trackbars")
    vMax = cv2.getTrackbarPos("vMax", "Trackbars")
    
    lowerArr = np.array([hMin, sMin, vMin])
    upperArr = np.array([hMax, sMax, vMax])
    mask = cv2.inRange(img, lowerArr, upperArr)
    
    cv2.rectangle(mask, (300,100), (500,300), (255,255,255), 1)
    
    cropped = mask[100:300, 300:500]
    cropped = cv2.cvtColor(cropped,cv2.COLOR_GRAY2RGB)
    cropped = cv2.resize(cropped, (224, 224))
    
    cropped = image.img_to_array(cropped)
    cropped = np.expand_dims(cropped, axis=0)
    cropped = keras.applications.mobilenet.preprocess_input(cropped)
    prediction = model.predict(cropped)
    fist = prediction[0][0]
    palm = prediction[0][1]
    peace = prediction[0][2]
    
    cv2.line(img, (20,20), (int(fist*500),20), (255,255,255), 8)
    cv2.line(img, (20,40), (int(palm*500),40), (255,255,255), 8)
    cv2.line(img, (20,60), (int(peace*500),60), (255,255,255), 8)
    
    fistP = int(fist*100)
    palmP = int(palm*100)
    peaceP = int(peace*100)
    
    cv2.putText(img, str(fistP)+"%", (450,20), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,255), 1)
    cv2.putText(img, str(palmP)+"%", (450,40), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,255), 1)
    cv2.putText(img, str(peaceP)+"%", (450,60), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,255), 1)
    
    cv2.putText(img, "fist", (485,20), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,255), 1)
    cv2.putText(img, "palm", (485,40), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,255), 1)
    cv2.putText(img, "peace", (485,60), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,255), 1)
    
    arr = [fist, palm, peace]

    ind = arr.index(max(arr))
    
    if ind==1:
        cv2.putText(img, "palm", (50,100), cv2.FONT_HERSHEY_COMPLEX, 1, (144,0,255), 1)
    elif ind==2:
        cv2.putText(img, "peace", (50,100), cv2.FONT_HERSHEY_COMPLEX, 1, (144,0,255), 1)
    else:
        cv2.putText(img, "fist", (50,100), cv2.FONT_HERSHEY_COMPLEX, 1, (144,0,255), 1)    
        
    cv2.imshow('1', img)
    cv2.imshow('2', mask)    
        
    inp = cv2.waitKey(1) & 0xFF
    if inp==ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break